In [1]:
import cv2
import os
import numpy as np
import datetime
from imutils.video import VideoStream
from datetime import timedelta
from tensorflow.keras.models import load_model
from openpyxl import Workbook
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:


# Define paths for data collection and model storage
data_path = "face_dataset"  # Replace with your dataset path
model_path = "face_recognition_model.h5"  # Trained model output path

names = []        # Initialize an empty list to store names
if os.path.exists('face_dataset') :
    names.extend(os.listdir('face_dataset'))
        
label_encoder = LabelEncoder()

def collect_facial_data(person_name):
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0)  # Use 0 for default webcam

    # Create a directory for the person's data if it doesn't exist
    os.makedirs(os.path.join(data_path, person_name), exist_ok=True)
    if os.path.exists('face_dataset') :
        names.extend(os.listdir('face_dataset'))
    num_samples = 0

    while True:
        ret, frame = cap.read()
        frame = cv2.flip(frame,1)
        
#         color = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        faces = face_cascade.detectMultiScale(frame, 1.8, 5)

        for (x, y, w, h) in faces:
#             roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y-68:y+h+18, x-28:x+w+38]

            cv2.rectangle(frame, (x-30, y-70), (x+w+40, y+h+20), (0, 255, 0), 2)
            num_samples += 1
            cv2.imwrite(os.path.join(data_path, person_name, f"{num_samples}.jpg"), roi_color)


        cv2.putText(frame, f"Samples Collected: {num_samples}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow('Collecting Facial Data', frame)
        key = cv2.waitKey(1) & 0xFF
        
        # Quit on 'q' press or after collecting enough samples
        if key == ord('q') or num_samples >= 100:
            break
            
    # Add the new person's name to the list
    names.append(person_name)

    # Update the label encoder with the expanded list of names
    
    label_encoder.fit(names)
    
    cap.release()
    cv2.destroyAllWindows()



# ... (other code)
label_encoder.fit(names)

# collect_facial_data('Anshu')
# 

LabelEncoder()

In [ ]:
# Function to train the CNN model
def train_cnn_model():
   
    # Load images and labels from data directory
    images = []
    labels = []
    for person_name in os.listdir(data_path):
        for filename in os.listdir(os.path.join(data_path, person_name)):
            img_path = os.path.join(data_path, person_name, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (200, 200))  # Resize images for CNN input
            images.append(img)
            labels.append(person_name)

    # Convert images and labels to NumPy arrays
    images = np.array(images, dtype=np.float32) / 255.0
    labels = np.array(labels)

    # One-hot encode labels for categorical crossentropy loss
   
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    onehot_encoder = OneHotEncoder(sparse=False)
    labels = onehot_encoder.fit_transform(labels.reshape(-1, 1))

    # Build and train the CNN model
    model = Sequential()
    
    # First convolutional layer     
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same',activation='relu', kernel_initializer='he_uniform', input_shape=(200, 200, 3)))
    model.add(MaxPooling2D((2, 2)))
    
    #Second Convolultional layer 
    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    
    #Third convolutional layer     
    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling2D((2, 2)))
    
    # Dropout for regularization
    model.add(Dropout(0.25))
    
    # Flatten the output of the convolutional layers
    model.add(Flatten())
    
    # Dense layers for classification
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Output layer with num_classes

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Split data into training and validation sets (optional)
    from sklearn.model_selection import train_test_split
    X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

    # Train the model (adjust epochs based on dataset size and computational resources)
    model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))  # Consider validation for overfitting
    model.summary()
    # Save the trained model
    model.save(model_path)  # Use the defined model_path

# Train the CNN model
train_cnn_model()

print("Model training complete!")

In [ ]:
# confusion matrix

In [2]:
# Define paths and variables
data_path = "face_dataset"
model_path = "face_recognition_model.h5"
attendance_file = "attendance.xlsx"
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
recorded_names = set() # Create an empty set to store already recorded names


# import faceliveness  # Assuming you've installed FaceLiveness

# Load a liveness detection model
# livemodel = faceliveness.models.DepthLiveness()




def is_live(frame, prev_frame, threshold=10):
    # Implement your logic here (e.g., compare difference between frames, track eye regions)
    # You can use libraries like OpenCV's SURF features for motion detection
    # Return True if liveness is detected, False otherwise
    
    # ... (Logic to capture frames or depth data)

#     if livemodel.is_live(frames_or_depth_data):
#         return True  # Likely live
#     else:
#         return False  # Might be spoofing
    return True  # Replace with your implementation


# Function to recognize face and mark attendance
def recognize_and_mark_attendance(model):
    # Load the trained CNN model
    model = load_model(model_path)
    
    global label_encoder  # Access the global label encoder
    # Load the label encoder if not already loaded
       
    # Create an Excel workbook for attendance
    wb = Workbook()
    ws = wb.active
    ws.append(["Date", "Time", "Name"])  # Header row

    # Start video stream
    vs = VideoStream(src=0).start()
    while True:
        frame = vs.read()
#         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        faces = face_cascade.detectMultiScale(frame, 1.8, 5)
        num_samples = 0
        prev_frame = None  # Store the previous frame for comparison
        blink_count = 0  # Track number of blinks within a timeframe
        start_time = None  # Time when collection starts
        
        for (x, y, w, h) in faces:
#           roi_gray = gray[y:y+h, x:x+w]  # Region of interest for face
            roi_color = frame[y-68:y+h+18, x-28:x+w+38]
            # Liveness detection (replace with your implementation)
            prev_frame = frame  # Assuming you have a mechanism to store the previous frame
            if not is_live(roi_color, prev_frame):
                cv2.putText(frame, "Not Live Face Detected", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                continue

            # Resize for model input
            img = cv2.resize(roi_color, (200, 200))
            img = np.expand_dims(img, axis=0)  # Add a dimension for batch processing
            img = np.array(img, dtype=np.float32) / 255.0
        
            # Predict probability distribution for each class (person)
            predictions = model.predict(img)[0]

            # Find the class with the highest probability
            max_index = np.argmax(predictions)
            predicted_name = label_encoder.inverse_transform([max_index])[0]
            proba = predictions[max_index]

            # Display name and confidence level if probability is high enough
            if predicted_name in recorded_names and proba > 0.8:  # Check if already recorded
                print(f"{predicted_name} already marked attendance.") 
                cv2.putText(frame, f"{predicted_name} ({proba:.2f})", (x-30, y - 75),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            elif proba > 0.9:  # Adjust threshold based on your model's performance
                cv2.putText(frame, f"{predicted_name} ({proba:.2f})", (x-30, y - 75),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                recorded_names.add(predicted_name)
                # Mark attendance in Excel
                now = datetime.datetime.now()
                ws.append([now.strftime("%Y-%m-%d"), now.strftime("%H:%M:%S"), predicted_name])

            else:
                cv2.putText(frame, "Unknown", (x-30, y - 70),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

            cv2.rectangle(frame, (x-30, y-70), (x+w+20, y+h+40), (255, 0, 0), 2)

        cv2.imshow('Attendance System', frame)
        key = cv2.waitKey(1) & 0xFF

        # Save attendance data and quit on 'q' press
        if key == ord('q'):
            wb.save(attendance_file)
            break

    vs.stop()
    cv2.destroyAllWindows()


In [3]:
model= load_model

recognize_and_mark_attendance(model)

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [ ]:
pip install faceliveness